In [1]:
!pip install fvcore
!pip install open_clip_torch
# Cell 1: clone MaCo repo (change URL if you want your fork)
!git clone https://github.com/RPIDIAL/CheXFound.git /kaggle/working/CheXFound
!gdown --folder https://drive.google.com/drive/folders/1GX2BWbujuVABtVpSZ4PTBykGULzrw806 -O /kaggle/working/CheXFound_Model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=e7607b24d92d657478820c8dfef6e3b3cbcc63cb9050d78efce0958a0815dafd
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=bf9c9bcdeb2cb93e839f53d6dd79bbc3b50ec53a9437836097090a719ef9ab84
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Cell 3: add repo to PYTHONPATH in the current notebook (no restart)
import sys
repo_path = "/kaggle/working/CheXFound"
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)



In [3]:
import os
os.chdir('/kaggle/working/CheXFound')  # Change this path to the repository path
import json
import torch
import argparse
import pandas as pd
from PIL import Image
from chexfound.eval.setup import setup_and_build_model
from chexfound.data.transforms import make_classification_eval_transform
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from chexfound.eval.utils import extract_hyperparameters_from_model
from chexfound.eval.classification.utils import setup_glori
from fvcore.common.checkpoint import Checkpointer
from matplotlib.colors import ListedColormap

import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchvision.io as io
from sklearn.model_selection import StratifiedShuffleSplit, KFold
from torch.amp import autocast, GradScaler
from torchvision.transforms import autoaugment, transforms
from transformers import get_cosine_schedule_with_warmup
import copy

/kaggle/working/CheXFound/chexfound/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/kaggle/working/CheXFound/chexfound/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/kaggle/working/CheXFound/chexfound/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
2025-10-01 10:07:05.441912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759313225.635546      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759313225.693257      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for p

In [4]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
target_class = 'Cardiomegaly'
assert target_class in ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

base_dir = '/kaggle/working/CheXFound_Model/'  # Change to the directory storing checkpoints and configuration files

config_file = base_dir + 'config.yaml'
pretrained_weights = base_dir + 'teacher_checkpoint.pth'
classifier_fpath = base_dir + 'glori.pth'
classifier_json = base_dir + 'results_eval_linear.json'
output_dir = base_dir + 'example'
os.makedirs(output_dir, exist_ok=True)

In [6]:
parser = argparse.ArgumentParser()

parser.set_defaults(
    config_file=config_file,  # path to architecture configuration files
    pretrained_weights=None,
    output_dir=output_dir,
    opts=[],
    image_size=512,
    patch_size=16,
    n_register_tokens=4,
    n_last_blocks=4,
    return_class_token=True,
    num_classes=40,
    num_heads=8,
)
args, unknown = parser.parse_known_args()

In [7]:
# set up foundation model
base_model, autocast_dtype = setup_and_build_model(args)

# load checkpoint for foundation model
state_dict = torch.load(pretrained_weights)['teacher']
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('backbone'):
        ls = k.split('.')
        if 'blocks' in k:
            new_k = '.'.join([ls[1], *ls[3:]])
        else:
            new_k = '.'.join(ls[1:])
    else:
        new_k = k
    new_state_dict.update({new_k: v})

base_model.load_state_dict(new_state_dict, strict=False)

[rank0]:[W1001 10:07:21.707765751 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.


I20251001 10:07:21 19 chexfound config.py:60] git:
  sha: a4bac4956f3eeba4a8b2c15410c600536c7b0bce, status: has uncommitted changes, branch: main

I20251001 10:07:21 19 chexfound config.py:61] config_file: /kaggle/working/CheXFound_Model/config.yaml
image_size: 512
n_last_blocks: 4
n_register_tokens: 4
num_classes: 40
num_heads: 8
opts: ['train.output_dir=/kaggle/working/CheXFound_Model/example']
output_dir: /kaggle/working/CheXFound_Model/example
patch_size: 16
pretrained_weights: None
return_class_token: True
I20251001 10:07:21 19 chexfound config.py:26] sqrt scaling learning rate; base: 0.0002, new: 2.3385358667337133e-05
I20251001 10:07:21 19 chexfound config.py:33] MODEL:
  WEIGHTS: ''
compute_precision:
  grad_scaler: true
  teacher:
    backbone:
      sharding_strategy: SHARD_GRAD_OP
      mixed_precision:
        param_dtype: fp16
        reduce_dtype: fp16
        buffer_dtype: fp32
    dino_head:
      sharding_strategy: SHARD_GRAD_OP
      mixed_precision:
        param_dty

_IncompatibleKeys(missing_keys=[], unexpected_keys=['dino_head.mlp.0.weight', 'dino_head.mlp.0.bias', 'dino_head.mlp.2.weight', 'dino_head.mlp.2.bias', 'dino_head.mlp.4.weight', 'dino_head.mlp.4.bias', 'dino_head.last_layer.weight_g', 'dino_head.last_layer.weight_v', 'ibot_head.mlp.0.weight', 'ibot_head.mlp.0.bias', 'ibot_head.mlp.2.weight', 'ibot_head.mlp.2.bias', 'ibot_head.mlp.4.weight', 'ibot_head.mlp.4.bias', 'ibot_head.last_layer.weight_g', 'ibot_head.last_layer.weight_v'])

In [8]:
# set up glori head
log_json = classifier_json
with open(log_json, 'r') as f:
    content = f.read().split('\n')[-3]
    data = json.loads(content)
best_classifier_str = data['best_classifier']['name']
hyperparameters = extract_hyperparameters_from_model(best_classifier_str)
learning_rate, avgpool, block = hyperparameters["lr"], hyperparameters["avgpool"], hyperparameters["blocks"]

sample_input = torch.randn(1, 3, 512, 512).cuda()
with torch.no_grad():
    sample_output = base_model.get_intermediate_layers(sample_input, n=args.n_last_blocks, return_class_token=True)

glori, _ = setup_glori(
    sample_output=sample_output,
    n_last_blocks_list=block,
    learning_rates=learning_rate,
    avgpools=avgpool,
    num_classes=args.num_classes,
    multiview=False,
    decoder_dim=768,
    cat_cls=False,
)

# load checkpoint for glori classifier
checkpointer = Checkpointer(glori)
checkpointer.load(classifier_fpath)
print('done')

I20251001 10:07:28 19 fvcore.common.checkpoint checkpoint.py:150] [Checkpointer] Loading from /kaggle/working/CheXFound_Model/glori.pth ...
W20251001 10:07:29 19 fvcore.common.checkpoint checkpoint.py:352] The checkpoint state_dict contains keys that are not used by the model:
  classifiers_dict.linear:blocks=4:avgpool=False:lr=0_0000100000.decoder.{duplicate_pooling, duplicate_pooling_bias}
  classifiers_dict.linear:blocks=4:avgpool=False:lr=0_0000100000.decoder.layers.0.norm1.{bias, weight}
  classifiers_dict.linear:blocks=4:avgpool=False:lr=0_0000100000.decoder.layers.0.multihead_attn.{in_proj_bias, in_proj_weight}
  classifiers_dict.linear:blocks=4:avgpool=False:lr=0_0000100000.decoder.layers.0.multihead_attn.out_proj.{bias, weight}
  classifiers_dict.linear:blocks=4:avgpool=False:lr=0_0000100000.decoder.layers.0.linear1.{bias, weight}
  classifiers_dict.linear:blocks=4:avgpool=False:lr=0_0000100000.decoder.layers.0.linear2.{bias, weight}
  classifiers_dict.linear:blocks=4:avgpool=

In [9]:
glori

DistributedDataParallel(
  (module): AllClassifiers(
    (classifiers_dict): ModuleDict(
      (linear:blocks=4:avgpool=False:lr=0_0005000000): GLoRI(
        (decoder): TransformerDecoder(
          (layers): ModuleList(
            (0): TransformerDecoderLayerOptimal(
              (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (dropout1): Dropout(p=0.1, inplace=False)
              (dropout2): Dropout(p=0.1, inplace=False)
              (dropout3): Dropout(p=0.1, inplace=False)
              (multihead_attn): MultiheadAttention(
                (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
              )
              (linear1): Linear(in_features=768, out_features=2048, bias=True)
              (linear2): Linear(in_features=2048, out_features=768, bias=True)
              (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (norm3

In [10]:
# ---------- Fix decoder internals (resize biases / num_classes) ----------
import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TARGET_CLASSES = 14
ORIG_CLASSES = 40

# get classifier same way as before
glori_wrapped = glori
glori_module = getattr(glori_wrapped, "module", glori_wrapped)
classifier_key = next(iter(glori_module.classifiers_dict.keys()))
classifier = glori_module.classifiers_dict[classifier_key]
decoder = getattr(classifier, "decoder", None)
if decoder is None:
    raise RuntimeError("Decoder not found in classifier; cannot resize num_classes. Print classifier to inspect.")

print("Decoder found:", type(decoder))
print("Decoder.num_classes (before):", getattr(decoder, "num_classes", None))

# 1) set decoder.num_classes
decoder.num_classes = TARGET_CLASSES
print("Set decoder.num_classes ->", decoder.num_classes)

# 2) replace duplicate_pooling_bias if exists (common in this implementation)
if hasattr(decoder, "duplicate_pooling_bias"):
    old = decoder.duplicate_pooling_bias
    print("Old duplicate_pooling_bias shape:", tuple(old.shape) if isinstance(old, torch.Tensor) else type(old))
    # create new bias param
    new_bias = nn.Parameter(torch.zeros(TARGET_CLASSES, dtype=torch.float32, device=device))
    # init small (optional)
    nn.init.constant_(new_bias, 0.0)
    # assign (if module originally on CPU, move)
    decoder.duplicate_pooling_bias = new_bias
    print("Replaced duplicate_pooling_bias with shape:", decoder.duplicate_pooling_bias.shape)
else:
    print("No attribute duplicate_pooling_bias found on decoder — fine.")

# 3) scan decoder for any 1D parameters/buffers whose length equals ORIG_CLASSES and replace them (safe heuristic)
replaced = []
for name, param in list(decoder.named_parameters()):
    if param.dim() == 1 and param.shape[0] == ORIG_CLASSES:
        print(f"Resizing param: {name} shape {param.shape} -> {TARGET_CLASSES}")
        parent = decoder
        parts = name.split('.')
        for p in parts[:-1]:
            parent = getattr(parent, p)
        attr = parts[-1]
        new_p = nn.Parameter(torch.zeros(TARGET_CLASSES, dtype=param.dtype, device=device))
        setattr(parent, attr, new_p)
        replaced.append(name)

# also check buffers (non-parameter tensors)
for name, buf in list(decoder.named_buffers()):
    if buf is None:
        continue
    if buf.ndim == 1 and buf.shape[0] == ORIG_CLASSES:
        print(f"Resizing buffer: {name} shape {buf.shape} -> {TARGET_CLASSES}")
        parent = decoder
        parts = name.split('.')
        for p in parts[:-1]:
            parent = getattr(parent, p)
        attr = parts[-1]
        # assign as buffer
        decoder.register_buffer(attr, torch.zeros(TARGET_CLASSES, dtype=buf.dtype, device=device))
        replaced.append("buffer:"+name)

print("Replaced parameter/buffer names:", replaced)

# 4) move whole glori to device again and do a quick forward test
glori_wrapped.to(device)
print("Moved glori to device.")

# Ensure backbone frozen if desired
for p in base_model.parameters():
    p.requires_grad = False

Decoder found: <class 'chexfound.eval.classification.glori.TransformerDecoder'>
Decoder.num_classes (before): 40
Set decoder.num_classes -> 14
Old duplicate_pooling_bias shape: (40,)
Replaced duplicate_pooling_bias with shape: torch.Size([14])
Replaced parameter/buffer names: []
Moved glori to device.


In [11]:
# 6) Wrap backbone + new glori into a training model
class CheXFoundWithGLoRIHead(nn.Module):
    def __init__(self, backbone, glori_module, args):
        super().__init__()
        self.backbone = backbone
        self.glori = glori_module
        self.args = args
    def forward(self, x):
        # get intermediate features and pass to glori
        feats = self.backbone.get_intermediate_layers(x, n=self.args.n_last_blocks, return_class_token=self.args.return_class_token)
        logits_dict = self.glori([feats], return_attention=False)
        key = list(logits_dict.keys())[0]
        return logits_dict[key]

In [12]:
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

cv2.setNumThreads(0)

class ChestXrayDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.label_cols = [
            "Atelectasis","Cardiomegaly","Consolidation","Edema",
            "Enlarged Cardiomediastinum","Fracture","Lung Lesion",
            "Lung Opacity","No Finding","Pleural Effusion","Pleural Other",
            "Pneumonia","Pneumothorax","Support Devices",
        ]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["Image_name"])

        # Fast read with OpenCV. It may return None if file missing -> handle.
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path}")

        # If grayscale (H, W), convert to 3-channel
        if img.ndim == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        elif img.shape[2] == 4:
            # if RGBA, convert to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        # OpenCV loads BGR -> convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image=img)
            img = augmented["image"]

        labels = torch.tensor(row[self.label_cols].values.astype("float32"))
        return img, labels

# choose img_size once
img_size = 512
size_tuple = (img_size, img_size)   # IMPORTANT: albumentations expects a tuple

train_tfms = A.Compose([
    A.RandomResizedCrop(size=size_tuple, scale=(0.85, 1.0), ratio=(0.9, 1.1)),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=12, p=0.4),
    A.CLAHE(p=0.3),
    A.RandomBrightnessContrast(p=0.3),
    A.CoarseDropout(max_holes=1, max_height=20, max_width=20, p=0.2),
    A.ShiftScaleRotate(shift_limit=0.03, scale_limit=0.05, rotate_limit=0, p=0.25),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_tfms = A.Compose([
    A.Resize(height=img_size, width=img_size),   # pass height & width
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

W20251001 10:07:29 19 py.warnings warnings.py:110] /tmp/ipykernel_19/1834097517.py:59: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, max_height=20, max_width=20, p=0.2),

W20251001 10:07:29 19 py.warnings warnings.py:110] /usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)



In [13]:
from torch.amp import autocast, GradScaler
from sklearn.metrics import roc_auc_score

scaler = GradScaler(device="cuda")

#EMA
# -----------------------------
# Train One Epoch (with AMP)
# -----------------------------
def train_one_epoch(model, loader, optimizer, criterion, ema=None, grad_clip=1.0):
    model.train()
    running_loss = 0.0
    
    progress_bar = tqdm(loader, desc="[Train]")
    for imgs, labels in progress_bar:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE).float()

        optimizer.zero_grad()
        with autocast(device_type="cuda" ):  # mixed precision forward
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        # scaler.unscale_(optimizer)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        
        if ema is not None:
            ema.update(model)
    #     if ema is not None:
    #         # support DataParallel-wrapped EMA as well as normal EMA
    #         ema_core = getattr(ema, "module", ema)
    #         ema_core.update(model)

        running_loss += loss.item() * imgs.size(0)
    return running_loss / len(loader.dataset)


# -----------------------------
# Validation (macro + per-label AUROC)
# -----------------------------
def validate(model, loader, criterion):
    
    model = nn.DataParallel(model)
    model.eval()
    running_loss = 0.0
    all_labels, all_outputs = [], []

    progress_bar = tqdm(loader, desc="[Val]")
    with torch.no_grad():
        for imgs, labels in progress_bar:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE).float()
            with autocast(device_type="cuda" ):
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            running_loss += loss.item() * imgs.size(0)
            all_labels.append(labels.cpu().numpy())
            all_outputs.append(outputs.sigmoid().cpu().numpy())

    all_labels = np.vstack(all_labels)
    all_outputs = np.vstack(all_outputs)

    per_label_auc = {}
    for i, label in enumerate(loader.dataset.label_cols):
        try:
            per_label_auc[label] = roc_auc_score(all_labels[:, i], all_outputs[:, i])
        except ValueError:
            per_label_auc[label] = np.nan

    try:
        #macro_auc = roc_auc_score(all_labels, all_outputs, average="macro", multi_class="ovo")
        macro_auc = roc_auc_score(all_labels, all_outputs, average="macro")
    except ValueError:
        macro_auc = np.nan

    return running_loss / len(loader.dataset), macro_auc, per_label_auc


# Learning rate scheduler
def get_scheduler(optimizer, num_training_steps):
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(num_training_steps * 0.1),
        num_training_steps=num_training_steps,
    )
    return scheduler

In [14]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # pt = p if target=1, 1-p otherwise
        focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

class WeightedFocalBCELoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, pos_weights=None):
        super().__init__()
        self.focal = FocalLoss(alpha, gamma)
        self.bce = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
        self.alpha = 0.7  # Weight between focal and BCE
        
    def forward(self, inputs, targets):
        focal_loss = self.focal(inputs, targets)
        bce_loss = self.bce(inputs, targets)
        return self.alpha * focal_loss + (1 - self.alpha) * bce_loss

# Calculate better class weights
def calculate_effective_weights(df, label_cols, beta=0.999):
    """Calculate more robust class weights using effective number of samples"""
    n = len(df)
    weights = []
    for col in label_cols:
        pos_count = df[col].sum()
        neg_count = n - pos_count
        
        # Effective number of samples (from Class-Balanced Loss paper)
        eff_pos = (1 - beta**pos_count) / (1 - beta) if pos_count > 0 else 0
        eff_neg = (1 - beta**neg_count) / (1 - beta) if neg_count > 0 else 0
        
        weight = eff_neg / (eff_pos + 1e-8)  # Avoid division by zero
        weights.append(min(weight, 10.0))  # Cap extreme weights
    
    return torch.tensor(weights).float()

In [15]:
class ModelEMA:
    def __init__(self, model, decay=0.9997, device=None):
        # copy *unwrapped* module (so EMA keys match saved/loaded model keys)
        self.decay = decay
        self.device = device
        self.ema = copy.deepcopy(get_model(model)).eval()
        if device is not None:
            self.ema.to(device)
        for p in self.ema.parameters():
            p.requires_grad_(False)

    def update(self, model):
        src_state = get_model(model).state_dict()
        with torch.no_grad():
            for k, v in self.ema.state_dict().items():
                model_v = src_state[k].detach().to(v.device)
                v.copy_(v * self.decay + (1.0 - self.decay) * model_v)

    def state_dict(self):
        return self.ema.state_dict()

In [16]:
def get_model(module):
    """Return the underlying model (unwrap DataParallel / DDP)."""
    return module.module if hasattr(module, "module") else module

In [17]:
EPOCHS=6

# -----------------------------
# Run Training
# -----------------------------

# # Prepare for cross-validation
# kf = KFold(n_splits=5, shuffle=True, random_state=32)
fold_results = []

df_all = pd.read_csv('/kaggle/input/grand-xray-slam-division-a/train1.csv')
df_all = df_all[~df_all['Image_name'].isin([
    '00043046_001_001.jpg',
    '00052495_001_001.jpg',
    '00056890_001_001.jpg'
])]

label_cols = [
            "Atelectasis",
            "Cardiomegaly",
            "Consolidation",
            "Edema",
            "Enlarged Cardiomediastinum",
            "Fracture",
            "Lung Lesion",
            "Lung Opacity",
            "No Finding",
            "Pleural Effusion",
            "Pleural Other",
            "Pneumonia",
            "Pneumothorax",
            "Support Devices",
        ]

    

best_auc = -1.0   # track best AUC
best_loss = float("inf")
# Create datasets
train_df = df_all.copy()
#val_df = df_all.iloc[val_idx].copy()
# -----------------------------
# Model: EVAX base
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# #model = CheXFound_Model(model)
# model = train_model

# -----------------------------
# Loss & Optimizer
# -----------------------------

#criterion = FocalLoss(alpha=0.25, gamma=2.0)
pos_weights = calculate_effective_weights(train_df, label_cols, beta=0.9999)
criterion = WeightedFocalBCELoss(alpha=0.25, gamma=2.0, pos_weights=pos_weights.to(DEVICE))


# -------------------------
# SAFE unwrap & prepare model for multi-GPU
# -------------------------
print("Preparing model on device:", DEVICE, "GPUs:", torch.cuda.device_count())

# 1) If glori is a DDP wrapper, unwrap to get the real module
glori_wrapped = glori  # original object you loaded
if hasattr(glori_wrapped, "module"):
    print("Unwrapping glori from DDP wrapper -> using glori.module")
    glori_clean = glori_wrapped.module
else:
    glori_clean = glori_wrapped

# move glori_clean to CPU to avoid device mismatch while assembling
glori_clean = glori_clean.cpu()

# 2) Rebuild train_model with the unwrapped glori (fresh object)
train_model = CheXFoundWithGLoRIHead(backbone=base_model, glori_module=glori_clean, args=args)

#train_model = train_model.to('cpu')
# 5) Move model to device
train_model = train_model.to(DEVICE)

# Load checkpoint
model_ckpt = torch.load("/kaggle/input/all-data-chexfound-recs-div-a/CheXFound/model3.pth", map_location=DEVICE,weights_only=False )
train_model.load_state_dict(model_ckpt["model_state_dict"])




# 3) Freeze backbone, ensure glori params are trainable
for p in train_model.backbone.parameters():
    p.requires_grad = False
for p in train_model.glori.parameters():
    p.requires_grad = True


# 4) If you have multiple GPUs, wrap with nn.DataParallel (NOT DDP)
if torch.cuda.device_count() > 1:
    print("Wrapping train_model with nn.DataParallel for multi-GPU")
    train_model = nn.DataParallel(train_model)



# IMPORTANT: make sure the name used in the rest of your loop refers to the wrapped model
model = train_model   

#EMA
ema = ModelEMA(model, decay=0.99967)

# ema_ckpt = torch.load("/kaggle/input/all-data-chexfound-recs-div-a/CheXFound/ema_model_f3.pth",
#                       #map_location='cpu',
#                       weights_only=False )
ema.ema.load_state_dict(model_ckpt["ema_state_dict"])


# 6) Build optimizer from glori params safely (now use `model`)
def get_glori_params_from_model(m):
    core = getattr(m, "module", m)
    gl = getattr(core, "glori", None)
    if gl is None:
        raise RuntimeError("model has no .glori attribute; inspect model.")
    gl_core = getattr(gl, "module", gl)
    return [p for p in gl_core.parameters() if p.requires_grad]

glori_trainable = get_glori_params_from_model(model)
print("Trainable glori params count:", sum(p.numel() for p in glori_trainable))


optimizer = torch.optim.AdamW(glori_trainable, lr=2e-4, weight_decay=1e-5)
#load optimizer
optimizer.load_state_dict(model_ckpt["optimizer_state_dict"])

    
# Dataset
train_dataset = ChestXrayDataset(
    df=train_df,
    img_dir="/kaggle/input/600-p-div-a-data/train1_resized",
    transform=train_tfms
)

# # (for a real setup, you'd split train/val from train1.csv)
# val_dataset = ChestXrayDataset(
#     df=val_df,
#     img_dir="/kaggle/input/xray-data-div-b-600p-95/train2_resized",
#     transform=val_tfms
# )

# DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=16,   
    #num_workers=4,   
    pin_memory=True
)
# val_loader = DataLoader(
#     val_dataset,
#     batch_size=64,
#     shuffle=False,
#     num_workers=16,
#     #num_workers=4,
#     pin_memory=True
# )

for epoch in range(3,EPOCHS):
    
    print(f"Epoch {epoch+1}/{EPOCHS}")
    
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, ema)

    #if epoch >1:
            
    #val_loss, val_auc, per_label_auc = validate(ema.ema, val_loader, criterion)
    print(f"[Epoch {epoch+1}] TrainLoss={train_loss:.4f} ")

    # print(f"[Epoch {epoch+1}] TrainLoss={train_loss:.4f} | EMA ValLoss={val_loss:.4f} | AUC={val_auc:.4f}")
    # print("Per-label AUROC:", {k: f"{v:.3f}" for k, v in per_label_auc.items()})

    
    save_path = f"best_model_f{epoch+1}.pth"
    torch.save({
        "epoch": epoch+1,
        "model_state_dict": get_model(model).state_dict(),  
        "optimizer_state_dict": optimizer.state_dict()
    }, save_path)

    # also save EMA
    if ema is not None:
        torch.save({
            "epoch": epoch+1,
            "ema_state_dict": ema.state_dict()
        }, f"ema_model_f{epoch+1}.pth")

    print(f"✅ Saved new best model  -> {save_path}")



Preparing model on device: cuda GPUs: 2
Unwrapping glori from DDP wrapper -> using glori.module
Wrapping train_model with nn.DataParallel for multi-GPU
Trainable glori params count: 8723470
W20251001 10:07:46 19 py.warnings warnings.py:110] /usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

Epoch 4/6


[Train]: 100%|██████████| 1678/1678 [2:53:08<00:00,  6.19s/it]


[Epoch 4] TrainLoss=0.0966 
✅ Saved new best model  -> best_model_f4.pth
Epoch 5/6


[Train]: 100%|██████████| 1678/1678 [2:53:01<00:00,  6.19s/it]


[Epoch 5] TrainLoss=0.0954 
✅ Saved new best model  -> best_model_f5.pth
Epoch 6/6


[Train]: 100%|██████████| 1678/1678 [2:53:32<00:00,  6.21s/it]


[Epoch 6] TrainLoss=0.0944 
✅ Saved new best model  -> best_model_f6.pth


In [18]:
# save_path = f"best_model_f{epoch+1}.pth"
# torch.save({
#     "epoch": epoch+1,
#     "model_state_dict": get_model(model).state_dict(),  
#     "optimizer_state_dict": optimizer.state_dict()
# }, save_path)

# # also save EMA
# if ema is not None:
#     torch.save({
#         "epoch": epoch+1,
#         "ema_state_dict": ema.state_dict()
#     }, f"ema_model_f{epoch+1}.pth")

# print(f"✅ Saved model  ")